# Matrix Factorization

Matrix Factorization is a machine learning algorithm used in recommendation engines.

A familiar example of a recommendation engine is the "recommended products" section of a shopping site. From the past purchase history of user A, we can determine other users B who have similar purchasing tendencies. If user B has purchased a product in the past, there is a high possibility that user A will also want it, so the system encourages purchase by displaying it for A as a recommended product.

This time, we will use quantum annealing to perform Matrix Factorization.  

References: O’Malley, Daniel, et al. "Nonnegative/binary matrix factorization with a d-wave quantum annealer." PloS one 13.12 (2018): e0206653.

We use MovieLens dataset (https://grouplens.org/datasets/movielens/100k/).  
It is a dataset of 100,000 ratings (1-5 stars) by 943 users for 1682 movies.

First, load the dataset.

In [1]:
import pandas as pd
import numpy as np

dataDir = 'https://files.grouplens.org/datasets/movielens/'

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(dataDir + 'ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(dataDir + 'ml-100k/u.data', sep='\t', names=r_cols,encoding='latin-1')

#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv(dataDir + 'ml-100k/u.item', sep='|', names=i_cols,
encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_train = pd.read_csv(dataDir + 'ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv(dataDir + 'ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

The dataset is divided into two parts, one for training and one for test.  
The contents are as follows. The rating given by the user indicated by "user_id" to the movie indicated by "movie_id" is recorded as "rating".

In [2]:
ratings_train

,user_id,movie_id,rating,unix_timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
90565,943,1047,2,875502146
90566,943,1074,4,888640250
90567,943,1188,3,888640250
90568,943,1228,3,888640275


We construct matrix from the data.  
The row will be "user_id", the column will be "movei_id", and each element will be the "rating" corresponding to that row and column.

Here, from the viewpoint of problem size, only data up to 100 for both "user_id" and "movei_id" will be extracted, and the matrix will be size of 100 x 100.

In [3]:
user_num = 100 # Max.943
item_num = 100 # Max.1680

def df_to_matrix(df, user_num, item_num, row_name, col_name, values):
    df1 = df[df[row_name] <= user_num]
    df2 = df1[df1[col_name] <= item_num]
    for i in range(1, user_num+1):
        if len(df2.loc[(df2.loc[:,row_name]==i)]) == 0:
            df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)
    for i in range(1, item_num+1):
        if len(df2.loc[(df2.loc[:,col_name]==i)]) == 0:
            df2 = df2.append(pd.DataFrame([[1,i]], columns=[row_name, col_name])).fillna(0)
    
    return np.array(df2.pivot(index = row_name, columns =col_name, values = values).fillna(0))
   
R = df_to_matrix(ratings_train, user_num, item_num, 'user_id', 'movie_id', 'rating')
print(R.shape)

R_test = df_to_matrix(ratings_test, user_num, item_num, 'user_id', 'movie_id', 'rating')
print(R_test.shape)

/tmp/ipykernel_371/973954993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)
/tmp/ipykernel_371/973954993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)
/tmp/ipykernel_371/973954993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)
/tmp/ipykernel_371/973954993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)


(100, 100)


/tmp/ipykernel_371/973954993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)
/tmp/ipykernel_371/973954993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)
/tmp/ipykernel_371/973954993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)
/tmp/ipykernel_371/973954993.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[i,1]], columns=[row_name, col_name])).fillna(0)


(100, 100)


/tmp/ipykernel_371/973954993.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[1,i]], columns=[row_name, col_name])).fillna(0)
/tmp/ipykernel_371/973954993.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[1,i]], columns=[row_name, col_name])).fillna(0)
/tmp/ipykernel_371/973954993.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(pd.DataFrame([[1,i]], columns=[row_name, col_name])).fillna(0)


In [4]:
R

array([[5., 3., 4., ..., 4., 3., 5.],
       [4., 0., 0., ..., 0., 0., 5.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 3., ..., 5., 0., 5.],
       [0., 0., 0., ..., 0., 0., 0.]])

Matrixf Factorization decomposes such a matrix as shown in the figure below.

$$
V \approx WH \\
$$

<img src="img/212_img.png" width="50%">

"features" represents the features that the user and the movie have respectively.  
A feature value indicates the preferences and tendencies of a user, for example, "the user likes action movies".  The same can be said for movie features, such as "action movie or not".  
Formulated in this way, we expect that users who like action movies will give high ratings to action movies they have not yet seen.  

Matrix Factorization finds $W, H$ that satisfy $V \approx WH $ with as much accuracy as possible.  
In particular, the problem where $W$ is positive and $H$ is restricted to binary (0 or 1) values is called Nonnegative/Binary Matrix Factorization (NBMF).  

NBMF repeatedly updates $W, H$ through the following optimization.

$$
W = \mathrm{argmin}_{X \in \mathbb{R}+^{M\times K}} \| V - XH \|_F  + \alpha \| X \|_F
$$

$$
H = \mathrm{argmin}_{X \in \{0,1\}^{K\times N} } \|V - WX \|_F
$$

Furthermore, each column $i$ of $H$ ($H_i$) can be optimized independently as follows

$$
H_i = \mathrm{argmin}_{q \in \{0,1\}^k } \|V_i-Wq \|_F
$$

$\|\cdot \|_F$ denotes the Frobenius norm.  
Since $H$ was assumed to be binary, we can take advantage of quantum annealing to find $q$.  
Specifically, it results in a linear least squares problem, and the following QUBO formulation is known.  

$$
f(q) = \sum_i a_i q_i + \sum_{i<j} b_{ij}q_i q_j  \\
a_j = \sum_k W_{kj}(W_{kj} - 2V_{ki}) \\
b_{jk} = 2\sum_l W_{lj}W_{lk}
$$


Reference: Borle, Ajinkya, and Samuel J. Lomonaco. "Analyzing the quantum annealing approach for solving linear least squares problems." International Workshop on Algorithms and Computation. Springer, Cham, 2019.

### Implementation

Let's implement it in blueqat.  
The following can be performed with both quantum annealing and QAOA.

In [5]:
from blueqat.utils import qaoa
from blueqat.pauli import qubo_bit as q

class MF():

    def __init__(self, V, K, alpha, iterations, Aneal_iteration, method):
        self.V = V
        self.num_users, self.num_items = V.shape
        self.K = K
        self.alpha = alpha
        self.iterations = iterations
        self.iteAneal = Aneal_iteration
        self.step = 5 # QAOA step
        self.method = method

    def train(self):
        # Initializing user-feature and movie-feature matrix 
        self.b = np.mean(self.V[np.where(self.V != 0)]) # bias term
        self.W = np.abs((np.random.normal(scale=self.b/self.K, size=(self.num_users, self.K))))
        self.H = np.abs((np.random.normal(scale=1./2, size=(self.K, self.num_items))))

        self.predictedIndex = []
        self.itemVecs = []
        self.itemVecs_indices = []
        for j in range(self.num_items):
            self.itemVecs.append([])
            self.itemVecs_indices.append([])
            for i in range(self.num_users):
                if self.V[i, j] > 0:
                    self.itemVecs[j].append(self.V[i, j]) # jth column's non 0 elements 
                    self.itemVecs_indices[j].append(i) # jth column's non 0 indices
        
        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            #np.random.shuffle(self.samples)
            self.train_W()
            if self.method == 'QAOA':
                self.train_H_QAOA()
            elif self.method == 'Anneal':
                self.train_H_Anneal()
            aveError = self.aveError()
            if (i+1) % 1 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, aveError))
            if i>0 and np.abs(aveError - training_process[-1][1]) < aveError/50: # convergence
                break
            else:
                training_process.append((i, aveError))
        return training_process

    # Computing mean error
    def aveError(self):
        xs, ys = self.V.nonzero()
        predicted = self.full_matrix()
        error = 0
        l = len(xs)
        for x, y in zip(xs, ys):
            error += np.abs(self.V[x, y] - predicted[x, y])
        return error / l

    # Stochastic gradient descent to get optimized W matrix
    def train_W(self):
        self.Wdiff = np.zeros([self.num_users, self.K])
        for i in range(self.num_users):
            for q in range(self.K):
                for j in range(self.num_items):
                    if self.V[i][j] == 0:
                        continue
                    self.Wdiff[i][q] += 2 * (self.V[i][j] - np.dot(self.W[i,:], self.H[:,j])) * (-1 * self.H[q][j])
        self.W = np.abs(self.W - self.alpha * self.Wdiff)

    def train_H_QAOA(self):
        # use QAOA method
        for j in range(self.num_items): #jth column of V    
            # define QUBO hamiltonian and solve
            self.hamiltonian = 0
            for l in range(self.K):
                for m in range(l,self.K):
                    if l==m: # diagonal elements
                        for k in self.itemVecs_indices[j]: #jth columns's k's element
                            self.hamiltonian += np.float64(self.W[k][l] * (self.W[k][l] - 2 * self.V[k][j])) * q(l)
                    else:
                        for k in self.itemVecs_indices[j]:
                            self.hamiltonian += 2 * np.float64(self.W[k][l] * self.W[k][m]) * q(l) * q(m)
            result = qaoa(self.hamiltonian, step=2)
            b = result.circuit.run(shots=10)
            self.H[:,j] = list(sample = b.most_common(1)[0][0])
            
    def train_H_Anneal(self):
        # use Q-Anealing
        self.JthAneal = 0
        for j in range(self.num_items): #jth column of V
            for ite in range(self.iteAneal): # iteration of anealing
                # define QUBO and solve
                self.a = wq.Opt()
                self.a.qubo = l_2d_ok = [[0] * self.K for i in range(self.K)] # initialize qubo matrix
                for l in range(self.K):
                    for m in range(l,self.K):
                        if l==m: # diagonal elements
                            for k in self.itemVecs_indices[j]: #jth columns's k's element
                                self.a.qubo[l][m] += self.W[k][l] * (self.W[k][l] - 2 * self.V[k][j])
                        else:
                            for k in self.itemVecs_indices[j]:
                                self.a.qubo[l][m] += 2 * self.W[k][l] * self.W[k][m]
                
                tmp = self.a.sa()
                tmp_error = self.error_aneal(tmp)
                if ite == 0:
                    self.anealBit = tmp
                    min_error = self.error_aneal(tmp)
                else: # update or not
                    if tmp_error < min_error:
                        self.anealBit = tmp
            
            self.H[:,j] = self.anealBit
                
    def error_aneal(self, anealBit):
        F = 0
        for l in range(self.K):
            for m in range(l,self.K):
                if l==m: # diagonal elements
                    F += self.a.qubo[l][m] * anealBit[l]
                else:
                    F += self.a.qubo[l][m] * anealBit[l] * anealBit[m]
        return abs(F)        

    # Full user-movie rating matrix
    def full_matrix(self):
        return self.W.dot(self.H)

### Learning

We will do the learning.  
When the reduction in errors converges to a certain degree, learning is terminated.

In [6]:
np.random.seed = 42
mf = MF(R, K=5, alpha=0.01, iterations=20, Aneal_iteration=1, method='QAOA')
training_process = mf.train()
print()
print("W x H:")
print(mf.full_matrix())

/tmp/ipykernel_371/2473069155.py:81: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.hamiltonian += np.float64(self.W[k][l] * (self.W[k][l] - 2 * self.V[k][j])) * q(l)


TypeError: can't multiply sequence by non-int of type 'float'

### Evalation of prediction performance

Use the learned features of user and movie to predict user's ratings of movies they haven't seen yet.  
The combination of users and movies to be predicted is included in the test data, and the evaluation in the test data is taken as the correct answer.

In [ ]:
prediction = mf.full_matrix()

In [ ]:
predictedRating = []
testRating = []
score = [[],[],[],[],[]]

xs, ys = R_test.nonzero()

error_test = 0
for x,y in zip(xs, ys):
    predictedRating.append(prediction[x][y])
    testRating.append(R_test[x][y])
    score[int(R_test[x][y] - 1)].append(min(prediction[x][y],5))

    error_test += np.abs(R_test[x][y] - prediction[x][y])
error_test = error_test / len(xs)

Let's calculate the average predicted score for each of the test data with ratings of 1, 2, 3, 4, and 5.  

In [ ]:
print('Test error =', error_test)
print()
print('the average predicted score for each of the test data with ratings of 1, 2, 3, 4, and 5.')
print(np.mean(score[0]), '\n',
     np.mean(score[1]), '\n',
     np.mean(score[2]), '\n',
     np.mean(score[3]), '\n',
     np.mean(score[4]),)   

Overall, the test data and the prediction results show the same high/low evaluation trends.  
It seems that it is difficult to discriminate points 2, 3, and 4 with this model and learning setting, but it may be possible to discriminate points 1 and 5.  

From the above, we have implemented a recommendation algorithm using Matrix Factorization with Quantum Annealing/QAOA.